In [ ]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install transformers[sentencepiece]
from google.colab import output
from huggingface_hub import notebook_login
from huggingface_hub import list_models, HfApi, ModelFilter
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

output.clear()

In [ ]:
notebook_login()

In [ ]:
list_models()

In [ ]:
hf_api = HfApi()
models = hf_api.list_models(
    filter=ModelFilter(
        task=["text-generation", "conversational"]
    )
)

model_Ids = [m.modelId for m in models]

In [ ]:
import ipywidgets as widgets

model_picker = widgets.Dropdown(options=model_Ids, value='microsoft/DialoGPT-medium')

In [ ]:
model_name = model_picker.value
print(f"Selected model is: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
step = 0
user_input = input(">> User:")
while user_input.lower()!='exit':
    
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step = step + 1
    user_input = input(">> User:")